In [28]:
from tools import get_puzzle, show_puzzle
TODAY = 7
puzzle = get_puzzle(TODAY)
show_puzzle(puzzle)

## --- Day 7: No Space Left On Device ---


You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?


The device the Elves gave you has problems with more than just its communication system. You try to run a system update:


Error```
 $ system-update --please --pretty-please-with-sugar-on-top
 : No space left on device

```


Perhaps you can delete some files to make space for the update?


You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:


```
 $ cd /
 $ ls
 dir a
 14848514 b.txt
 8504156 c.dat
 dir d
 $ cd a
 $ ls
 dir e
 29116 f
 2557 g
 62596 h.lst
 $ cd e
 $ ls
 584 i
 $ cd ..
 $ cd ..
 $ cd d
 $ ls
 4060174 j
 8033020 d.log
 5626152 d.ext
 7214296 k

```


The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called/``. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.


Within the terminal output, lines that begin with$``are **commands you executed** , very much like some modern computers:


+ cd``means **change directory** . This changes which directory is the current directory, but the specific result depends on the argument:+ cd x``moves **in** one level: it looks in the current directory for the directory namedx``and makes it the current directory.
+ cd ..``moves **out** one level: it finds the directory that contains the current directory, then makes that directory the current directory.
+ cd /``switches the current directory to the outermost directory,/``.


+ ls``means **list** . It prints out all of the files and directories immediately contained by the current directory:+ 123 abc``means that the current directory contains a file namedabc``with size123``.
+ dir xyz``means that the current directory contains a directory namedxyz``.




Given the commands and output in the example above, you can determine that the filesystem looks visually like this:


```
 - / (dir)
   - a (dir)
     - e (dir)
       - i (file, size=584)
     - f (file, size=29116)
     - g (file, size=2557)
     - h.lst (file, size=62596)
   - b.txt (file, size=14848514)
   - c.dat (file, size=8504156)
   - d (dir)
     - j (file, size=4060174)
     - d.log (file, size=8033020)
     - d.ext (file, size=5626152)
     - k (file, size=7214296)

```


Here, there are four directories:/``(the outermost directory),a``andd``(which are in/``), ande``(which is ina``). These directories also contain files of various sizes.


Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the **total size** of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)


The total sizes of the directories above can be found as follows:


+ The total size of directorye``is **584** because it contains a single filei``of size 584 and no other directories.
+ The directorya``has total size **94853** because it contains filesf``(size 29116),g``(size 2557), andh.lst``(size 62596), plus filei``indirectly (a``containse``which containsi``).
+ Directoryd``has total size **24933642** .
+ As the outermost directory,/``contains every file. Its total size is **48381165** , the sum of the size of every file.


To begin, find all of the directories with a total size of **at most 100000** , then calculate the sum of their total sizes. In the example above, these directories area``ande``; the sum of their total sizes is95437``(94853 + 584). (As in this example, this process can count files more than once!)


Find all of the directories with a total size of at most 100000. **What is the sum of the total sizes of those directories?** 




In [29]:
test = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""".splitlines()

from  functools import reduce
class Dir (object):
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.dirs = dict()
        self.files = set()
        self.filessize = 0
    
    def add_dir(self, name):
        new_dir = Dir(name, self)
        if name in self.dirs.keys():
            raise ValueError(f'{name}  already exists in {self.name}')
        self.dirs[name] = new_dir
    
    def add_file(self, name, filesize):
        if name in self.files:
            raise ValueError(f'File {name} already exists in {self.name}')
        self.files.add(name)
        self.filessize += filesize
    
    def dir_size(self):
        sizechildren= 0
        if len(self.dirs)>0:
            sizechildren = reduce(lambda a, b: a+ b, map(lambda x: x.dir_size() ,self.dirs.values()) ) 
        return self.filessize + sizechildren  

class File (object):
    def __init__(self, name, size):
        self.name = name
        self.size = size


def build_tree(data):
    meta = Dir("ROOT", None)
    root = Dir("/", meta)

    meta.add_dir(root.name)
    
    current = meta
    for l in data:
        if l.startswith("$ ls"):
            pass
        elif l.startswith("$ cd "):
            nextdir = l[5:]
            if nextdir == "..":
                current = current.parent
            else:
                assert nextdir in current.dirs.keys()
                current = current.dirs[nextdir]
        elif l.startswith("dir "):
            current.add_dir(l[4:])
        else:
            size, foundfile = l.split()
            if foundfile not in current.files:
                current.add_file(foundfile, int(size))
    return meta
            
def total_of_folders_less_than_100K(folder):
    LESSTHAN = 100000
    folders_to_visit = set()
    folders_visited = set()
    folders_to_visit.add(folder)
    total = 0
    while len(folders_to_visit)> 0:
        visiting = folders_to_visit.pop()
        folders_visited.add(visiting)
        size = visiting.dir_size()

        if size < LESSTHAN:
            total += size
        for k,v in visiting.dirs.items():
            folders_to_visit.add(v)
    return total


assert total_of_folders_less_than_100K(build_tree(test)) == 95437

In [37]:
print (f"Solution 1: Size of all folders smaller than 100K is {total_of_folders_less_than_100K(build_tree(puzzle.data))}")

Solution 1: Size of all folders smaller than 100K is 1908462


In [35]:
FILESYSTEM_SIZE = 70000000
UPDATE_SIZE = 30000000

def size_of_smallest_folder_bigger_than(folder, bigger_than):
    AMOUNT_USED = folder.dir_size()
    folders_to_visit = set()
    folders_visited = set()
    folders_to_visit.add(folder)
    
    # we start with the biggest size possible on folder
    min_size_above_than = folder.dir_size()
    
    while len(folders_to_visit)> 0:
        visiting = folders_to_visit.pop()
        folders_visited.add(visiting)
        size = visiting.dir_size()
        if min_size_above_than > size > bigger_than:
            min_size_above_than = size
        for k,v in visiting.dirs.items():
            folders_to_visit.add(v)
    return min_size_above_than

test_rootfolder = build_tree(test)
test_space_needed = UPDATE_SIZE + test_rootfolder.dir_size() - FILESYSTEM_SIZE
assert size_of_smallest_folder_bigger_than(test_rootfolder, test_space_needed) == 24933642

In [36]:
rootfolder = build_tree(puzzle.data)
space_needed = UPDATE_SIZE + rootfolder.dir_size() - FILESYSTEM_SIZE
print (f"Solution 2: Deleting folder of size {size_of_smallest_folder_bigger_than(rootfolder, space_needed)} will provide more than the {space_needed} needed")


Solution 2: Deleting folder of size 3979145 will provide more than the 3956976 needed
